In [9]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster


In [11]:
!pip install seaborn

In [13]:
import seaborn as sns

In [19]:
trash_hauler_df = pd.read_csv('../data/trash_hauler_report_with_lat_lng.csv')

In [21]:
print(trash_hauler_df.head())

   Request Number Date Opened                               Request  \
0           25270   11/1/2017                      Trash - Backdoor   
1           25274   11/1/2017  Trash - Curbside/Alley Missed Pickup   
2           25276   11/1/2017  Trash - Curbside/Alley Missed Pickup   
3           25307   11/1/2017  Trash - Curbside/Alley Missed Pickup   
4           25312   11/1/2017  Trash - Curbside/Alley Missed Pickup   

                                         Description       Incident Address  \
0  house with the wheel chair ramp, they share dr...         3817 Crouch Dr   
1                           Curb/Trash miss Tuesday.  4028 Clarksville Pike   
2                           Curb/trash miss Tuesday.    6528 Thunderbird Dr   
3                                             missed   2603 old matthews rd   
4                  Missed the even side of the road.          604 croley dr   

   Zip Code Trash Hauler Trash Route  Council District  State Plan X  \
0   37207.0    RED RIVER  

In [23]:
print(trash_hauler_df.tail())

       Request Number Date Opened                               Request  \
20221          267125   11/1/2019  Trash - Curbside/Alley Missed Pickup   
20222          267126   11/1/2019  Trash - Curbside/Alley Missed Pickup   
20223          267130   11/1/2019  Trash - Curbside/Alley Missed Pickup   
20224          267134   11/1/2019  Trash - Curbside/Alley Missed Pickup   
20225          267137   11/1/2019  Trash - Curbside/Alley Missed Pickup   

                                             Description  \
20221                          MISSED...NEIGHBORS MISSED   
20222                                       entire alley   
20223                                     missed several   
20224  Caller stated trash was missed & were only pic...   
20225                     possibly others missed as well   

                                        Incident Address  Zip Code  \
20221                             2731 Murfreesboro Pike   37013.0   
20222  1621 Long Ave, Nashville, TN 37206, Unite

In [27]:
print(trash_hauler_df.shape)

(20226, 13)


In [29]:
print(trash_hauler_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20226 entries, 0 to 20225
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Request Number    20226 non-null  int64  
 1   Date Opened       20226 non-null  object 
 2   Request           20226 non-null  object 
 3   Description       20195 non-null  object 
 4   Incident Address  20217 non-null  object 
 5   Zip Code          20151 non-null  float64
 6   Trash Hauler      19325 non-null  object 
 7   Trash Route       19279 non-null  object 
 8   Council District  20177 non-null  float64
 9   State Plan X      20198 non-null  float64
 10  State Plan Y      20198 non-null  float64
 11  LONGITUDE         20198 non-null  float64
 12  LATITUDE          20198 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 2.0+ MB
None


In [41]:
trash_hauler_rename = trash_hauler_df.rename(columns={'Zip Code' : 'Zip', 'LONGITUDE': 'lng', 'LATITUDE' : 'lat'})

print(list(trash_hauler_rename.columns))

['Request Number', 'Date Opened', 'Request', 'Description', 'Incident Address', 'Zip', 'Trash Hauler', 'Trash Route', 'Council District', 'State Plan X', 'State Plan Y', 'lng', 'lat']


## Missed Trash Pickups

In this data question you will be working data of service request related to missed trash pickups from hubNashville, Metro Nashville government's comprehensive customer service system (https://hub.nashville.gov).

As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. 

One category of liquidated damages is related to chronic problems in any category of service at the same premises. 

A chronic problem is defined as more than one missed pickup for any address. 

The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

Total incidents with only 1 missed pickup: 9190

Total incidents of more than one missed pickup: 5830

After determining the total amount of damages, you can look at other questions:

* What other types of complaints are there?

Request
Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2629
Trash Collection Complaint               2312
Damage to Property                        257

* Are there any geospatial analysis you can do?  Which visualizations can you create?
* How do metro crews compare to the contractor's performance?
Trash Haulers Request Count for Each Company:
RED RIVER    14395
METRO         3512
WASTE IND     1350
Metro           68


* How much does each trash hauler owe?
* What were to total missed pickup by route?


In [43]:
trash_hauler_requests_type = trash_hauler_rename['Request'].value_counts()

print(trash_hauler_requests_type)

Request
Trash - Curbside/Alley Missed Pickup    15028
Trash - Backdoor                         2629
Trash Collection Complaint               2312
Damage to Property                        257
Name: count, dtype: int64


In [63]:
trash_hauler_name = trash_hauler_rename['Trash Hauler'].value_counts()

print(trash_hauler_name)

Trash Hauler
RED RIVER    14395
METRO         3512
WASTE IND     1350
Metro           68
Name: count, dtype: int64


In [67]:
trash_hauler_missed_pickup=trash_hauler_rename[trash_hauler_rename['Request']== 'Trash - Curbside/Alley Missed Pickup']

print(trash_hauler_missed_pickup.head())

   Request Number Date Opened                               Request  \
1           25274   11/1/2017  Trash - Curbside/Alley Missed Pickup   
2           25276   11/1/2017  Trash - Curbside/Alley Missed Pickup   
3           25307   11/1/2017  Trash - Curbside/Alley Missed Pickup   
4           25312   11/1/2017  Trash - Curbside/Alley Missed Pickup   
8           25330   11/1/2017  Trash - Curbside/Alley Missed Pickup   

                         Description               Incident Address      Zip  \
1           Curb/Trash miss Tuesday.          4028 Clarksville Pike  37218.0   
2           Curb/trash miss Tuesday.            6528 Thunderbird Dr  37209.0   
3                             missed           2603 old matthews rd  37207.0   
4  Missed the even side of the road.                  604 croley dr  37209.0   
8                            Missed.  4484 Lavergne Couchville Pike  37013.0   

  Trash Hauler Trash Route  Council District  State Plan X  State Plan Y  \
1    RED RIVER  

In [47]:
print(trash_hauler_missed_pickup.tail())

       Request Number Date Opened                               Request  \
20221          267125   11/1/2019  Trash - Curbside/Alley Missed Pickup   
20222          267126   11/1/2019  Trash - Curbside/Alley Missed Pickup   
20223          267130   11/1/2019  Trash - Curbside/Alley Missed Pickup   
20224          267134   11/1/2019  Trash - Curbside/Alley Missed Pickup   
20225          267137   11/1/2019  Trash - Curbside/Alley Missed Pickup   

                                             Description  \
20221                          MISSED...NEIGHBORS MISSED   
20222                                       entire alley   
20223                                     missed several   
20224  Caller stated trash was missed & were only pic...   
20225                     possibly others missed as well   

                                        Incident Address      Zip  \
20221                             2731 Murfreesboro Pike  37013.0   
20222  1621 Long Ave, Nashville, TN 37206, United 

In [65]:
trash_hauler_requests_type_check = trash_hauler_missed_pickup['Request'].value_counts()
print(trash_hauler_requests_type_check)

Request
Trash - Curbside/Alley Missed Pickup    15028
Name: count, dtype: int64


In [73]:
trash_hauler_address_count = trash_hauler_missed_pickup['Incident Address'].value_counts()

repeated_missed_pickup=trash_hauler_address_count[trash_hauler_address_count>1]

print(repeated_missed_pickup)

Incident Address
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
5135 Hickory Hollow Pkwy                                      17
802 Crescent Rd, Nashville, TN 37205, United States           17
1816 Jo Johnston Ave, Nashville, TN 37203, United States      16
607 Estes Rd, Nashville, TN 37215, United States              15
                                                              ..
3702 Hobbs Rd                                                  2
1114 Saunders Ave, Madison, TN 37115, United States            2
1824 Shackleford Rd B                                          2
4014 Dorcas Dr, Nashville, TN 37215, United States             2
109 Jocelyn Hills Rd                                           2
Name: count, Length: 2226, dtype: int64


In [93]:
trash_hauler_address_count = trash_hauler_missed_pickup['Incident Address'].value_counts()
repeated_addresses = trash_hauler_address_count[trash_hauler_address_count > 1].index

repeated_missed_pickups = trash_hauler_missed_pickup[trash_hauler_missed_pickup['Incident Address'].isin(repeated_addresses)]

print(repeated_missed_pickups.head())

    Request Number Date Opened                               Request  \
1            25274   11/1/2017  Trash - Curbside/Alley Missed Pickup   
12           25359   11/1/2017  Trash - Curbside/Alley Missed Pickup   
19           25454   11/1/2017  Trash - Curbside/Alley Missed Pickup   
24           25496   11/1/2017  Trash - Curbside/Alley Missed Pickup   
29           25511   11/1/2017  Trash - Curbside/Alley Missed Pickup   

                               Description       Incident Address      Zip  \
1                 Curb/Trash miss Tuesday.  4028 Clarksville Pike  37218.0   
12  Missed. Does not have access to alley.        830 Meridian St  37207.0   
19                                 Missed.        449 Westboro Dr  37209.0   
24                           Missed pickup     1815 WOODMONT BLVD  37215.0   
29                                 Missed.        259 Sunrise Ave  37211.0   

   Trash Hauler Trash Route  Council District  State Plan X  State Plan Y  \
1     RED RIVER      

In [115]:
count_check=repeated_missed_pickups['Incident Address'].value_counts()
print(count_check)
print(repeated_missed_pickups.count())

Incident Address
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
5135 Hickory Hollow Pkwy                                      17
802 Crescent Rd, Nashville, TN 37205, United States           17
1816 Jo Johnston Ave, Nashville, TN 37203, United States      16
607 Estes Rd, Nashville, TN 37215, United States              15
                                                              ..
208 Apollo Cir, Nashville, TN 37013, United States             2
4422 Winston Ave W, Nashville, TN 37211, United States         2
1265 Rural Hill Rd, Nashville, TN 37013, United States         2
1413 NAPA PT W, 37013                                          2
643 E CAMPBELL RD                                              2
Name: count, Length: 2226, dtype: int64
Request Number      5830
Date Opened         5830
Request             5830
Description         5822
Incident Address    5830
Zip                 5822
Trash Hauler        5534
Trash Route         5522
Council District    5825
Sta

In [111]:
trash_hauler_address_count = trash_hauler_missed_pickup['Incident Address'].value_counts()
single_incident_address = trash_hauler_address_count[trash_hauler_address_count == 1].index

one_missed_pickup = trash_hauler_missed_pickup[trash_hauler_missed_pickup['Incident Address'].isin(single_incident_address)]

print(one_missed_pickup.head())

   Request Number Date Opened                               Request  \
2           25276   11/1/2017  Trash - Curbside/Alley Missed Pickup   
3           25307   11/1/2017  Trash - Curbside/Alley Missed Pickup   
4           25312   11/1/2017  Trash - Curbside/Alley Missed Pickup   
8           25330   11/1/2017  Trash - Curbside/Alley Missed Pickup   
9           25331   11/1/2017  Trash - Curbside/Alley Missed Pickup   

                         Description               Incident Address      Zip  \
2           Curb/trash miss Tuesday.            6528 Thunderbird Dr  37209.0   
3                             missed           2603 old matthews rd  37207.0   
4  Missed the even side of the road.                  604 croley dr  37209.0   
8                            Missed.  4484 Lavergne Couchville Pike  37013.0   
9  Trash missed Tuesday again, ALLEY                1206 Ireland St  37208.0   

  Trash Hauler Trash Route  Council District  State Plan X  State Plan Y  \
2    RED RIVER  

In [113]:
print(one_missed_pickup.count())

Request Number      9190
Date Opened         9190
Request             9190
Description         9169
Incident Address    9190
Zip                 9148
Trash Hauler        8886
Trash Route         8881
Council District    9164
State Plan X        9180
State Plan Y        9180
lng                 9180
lat                 9180
dtype: int64
